In [130]:
import pandas as pd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from subprocess import Popen, PIPE
import os
import pathlib
import shutil
import os


In [131]:
flank_len = 50

In [132]:
#RICE

In [178]:
name = '96'
organism = 'rice'

In [179]:
path_genome = '/home/juan/Desktop/juan/bio/data/genomes/rice/IRGSP-1.0_genome.fasta'
path_gff = 'data/' + organism + '/' + name + '.filtered.gff3'
path_out = 'data/' + organism + '/' + name + '.filtered.fasta'
path_out_flank = 'data/' + organism + '/' + name + '.filtered.flank.fasta'
path_out_only_flank = 'data/' + organism + '/' + name + '.filtered.onlyflank.fasta'
c_dir  = 'data/' + organism + '/' + name + '_clus/'
pid = 0.3

In [ ]:
#WHEAT

In [205]:



name = 'minos'
name = '87481'
name = '109136'
name = '117188'

In [206]:
path_genome = '/home/juan/Desktop/juan/bio/data/IWGSC/42/Triticum_aestivum.IWGSC.dna.toplevel.fa'
path_gff = 'data/wheat/' + name + '.filtered.gff3'
path_out = 'data/wheat/' + name + '.filtered.fasta'
path_out_flank = 'data/wheat/' + name + '.filtered.flank.fasta'
path_out_only_flank = 'data/wheat/' + name + '.filtered.onlyflank.fasta'
c_dir  = 'data/wheat/' + name + '_clus/'
pid = 0.3

In [215]:
df = pd.read_csv(path_gff, sep='\t')
df.columns = ['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute']
ele_len = (len(df.index))
print(ele_len)

104


In [216]:
pd.set_option('display.max_colwidth', -1)


In [217]:
df.head(3)

,seqname,source,feature,start,end,score,strand,frame,attribute
0,1B,MITE,TE,5749364,5749453,.,+,.,MITE_T_109136|chr6B|164810296|164810388|CAG|97|F4960
1,1B,MITE,TE,61951691,61951779,.,+,.,MITE_T_109136|chr6B|164810296|164810388|CAG|97|F4960
2,1B,MITE,TE,97376094,97376181,.,+,.,MITE_T_109136|chr6B|164810296|164810388|CAG|97|F4960


In [218]:
fasta_seq = SeqIO.parse(path_genome, 'fasta')
handle_out = open(path_out, "w")
handle_out_flank = open(path_out_flank, "w")
handle_out_only_flank = open(path_out_only_flank, "w")
for record in fasta_seq:
    buffer_mites = []
    buffer_mites_flank = []
    buffer_mites_only_flank = []
    dff_extract = df[df.seqname == record.id]
    count = 0
    curr = 0
    total = len(dff_extract.index)
    print(record.id, len(dff_extract.index))
    clean_seq = ''.join(str(record.seq).splitlines())
    for k,v in dff_extract.iterrows():
        start = min(v.start,v.end)
        end = max(v.start,v.end)
        new_seq = clean_seq[start:end]
        id = v.attribute + "_"
        att = record.id + "_" + str(start) + '_' + str(end)
        seq = SeqRecord(Seq(new_seq), id=id, description=att)
        buffer_mites.append(seq)
        
        #add flanking
        start_f = max(start-flank_len,0)
        end_f = end + flank_len
        new_seq = clean_seq[start_f:end_f]
        id = v.attribute + "_"
        att = record.id + "_flank_" + str(start_f) + '_' + str(end_f)
        seq = SeqRecord(Seq(new_seq), id=id, description=att)
        buffer_mites_flank.append(seq)
        
        #only flanking 1
        start_o_f_1 = max(start-flank_len,0)
        end_o_f_1 = start
        new_seq = clean_seq[start_o_f_1:end_o_f_1]
        id = v.attribute + "_"
        att = record.id + "_flank_1_" + str(start_o_f_1) + '_' + str(end_o_f_1)
        seq = SeqRecord(Seq(new_seq), id=id, description=att)
        buffer_mites_only_flank.append(seq)
        
        #only flanking 2
        start_o_f_2 = end 
        end_o_f_2 = end+flank_len
        new_seq = clean_seq[start_o_f_2:end_o_f_2]
        id = v.attribute + "_"
        att = record.id + "_flank_2_" + str(start_o_f_2) + '_' + str(end_o_f_2)
        seq = SeqRecord(Seq(new_seq), id=id, description=att)
        buffer_mites_only_flank.append(seq)
        
        count += 1
    SeqIO.write(buffer_mites, handle_out, "fasta")
    SeqIO.write(buffer_mites_flank, handle_out_flank, "fasta")
    SeqIO.write(buffer_mites_only_flank, handle_out_only_flank, "fasta")
handle_out.close()
handle_out_only_flank.close()
handle_out_flank.close()

1A 0
1B 4
1D 5
2A 2
2B 11
2D 4
3A 1
3B 7
3D 2
4A 4
4B 7
4D 6
5A 2
5B 11
5D 3
6A 0
6B 12
6D 3
7A 6
7B 9
7D 1
Un 4


In [219]:
if os.path.isdir(c_dir):
    shutil.rmtree(c_dir)
pathlib.Path(c_dir).mkdir(parents=True, exist_ok=True)

In [220]:
#clusterize flankings
cmd_list = [
'./bin/vsearch-2.11.1/bin/vsearch',
'--cluster_fast', path_out_only_flank,
'--threads',str(2),
'--strand','both',
'--clusters', c_dir + "c_",
'--id', str(pid)]
p = Popen(cmd_list, stdout=PIPE, stderr=PIPE)
out,err = p.communicate()

In [221]:
files = [f for f in os.listdir(c_dir) if os.path.isfile(os.path.join(c_dir, f))]

In [222]:
(len(files) / 2) * 100 / ele_len

94.71153846153847